Import necesary libraries 

In [1]:
import os
import pandas as pd
import cv2
import random
from ultralytics import YOLO


Data Preperation 

only using the LARD_Train_Birk_LFST folder of data, since the data set is so large and will take to long to run. I will be dividing that data set into training, validation and test data

In [2]:
# Specify the training folder to use
root_folder = r'C:\Users\Annabelle\Desktop\runway-bounding-box-detection NEW'
train_folder = 'LARD_train_BIRK_LFST'

In [3]:
# Paths for images, metadata, and output
folder_path = os.path.join(root_folder, train_folder) #path to folder 'LARD_train_BIRK_LFST'
image_folder = os.path.join(folder_path, 'images') #path to images sub-folder in 'LARD_train_BIRK_LFST'
csv_path = os.path.join(folder_path, 'LARD_train_BIRK_LFST.csv')  # path to CSV file
output_folder = 'dataset_demo' #new folder for the sample of train & val images and labels taking for this
subset_size = 20  # Number of images to sample for train/val. doing small ammount to speed up demo/training time 

In [4]:
print("Images found in the folder:")
print(os.listdir(image_folder))  # List all images in the folder

# Confirm the number of images found
print(f"Total images found: {len(os.listdir(image_folder))}")


Images found in the folder:
['BIRK_01_500_000.jpeg', 'BIRK_01_500_001.jpeg', 'BIRK_01_500_002.jpeg', 'BIRK_01_500_003.jpeg', 'BIRK_01_500_004.jpeg', 'BIRK_01_500_005.jpeg', 'BIRK_01_500_006.jpeg', 'BIRK_01_500_007.jpeg', 'BIRK_01_500_008.jpeg', 'BIRK_01_500_009.jpeg', 'BIRK_01_500_010.jpeg', 'BIRK_01_500_011.jpeg', 'BIRK_01_500_012.jpeg', 'BIRK_01_500_013.jpeg', 'BIRK_01_500_014.jpeg', 'BIRK_01_500_015.jpeg', 'BIRK_01_500_016.jpeg', 'BIRK_01_500_017.jpeg', 'BIRK_01_500_018.jpeg', 'BIRK_01_500_019.jpeg', 'BIRK_01_500_020.jpeg', 'BIRK_01_500_021.jpeg', 'BIRK_01_500_022.jpeg', 'BIRK_01_500_023.jpeg', 'BIRK_01_500_024.jpeg', 'BIRK_01_500_025.jpeg', 'BIRK_01_500_026.jpeg', 'BIRK_01_500_027.jpeg', 'BIRK_01_500_028.jpeg', 'BIRK_01_500_029.jpeg', 'BIRK_01_500_030.jpeg', 'BIRK_01_500_031.jpeg', 'BIRK_01_500_032.jpeg', 'BIRK_01_500_033.jpeg', 'BIRK_01_500_034.jpeg', 'BIRK_01_500_035.jpeg', 'BIRK_01_500_036.jpeg', 'BIRK_01_500_037.jpeg', 'BIRK_01_500_038.jpeg', 'BIRK_01_500_039.jpeg', 'BIRK_01_50

In [5]:
# Create output directory structure
os.makedirs(f'{output_folder}/images/train', exist_ok=True)
os.makedirs(f'{output_folder}/images/val', exist_ok=True)
os.makedirs(f'{output_folder}/labels/train', exist_ok=True)
os.makedirs(f'{output_folder}/labels/val', exist_ok=True)

In [6]:
# Load the CSV metadata with semicolon separator
df = pd.read_csv(csv_path, sep=';')

In [7]:
# **New code to handle 'images/' prefix in CSV filenames**
df['image'] = df['image'].str.replace('images/', '', regex=False).str.strip()

# Verify the adjustment by printing the first few filenames
print("Adjusted Filenames from the CSV:")
print(df['image'].head())  # Confirm that 'images/' prefix is removed

# Collect all images from the folder and randomly sample a subset
images = [img for img in os.listdir(image_folder) if img.endswith(('.jpg', '.jpeg'))]
print(f"Total images found in folder: {len(images)}")

# Check for missing images
missing_files = [img for img in df['image'] if img not in images]
if missing_files:
    print(f"These images are listed in the CSV but not found in the folder: {missing_files}")
else:
    print("All CSV images match with folder images!")

Adjusted Filenames from the CSV:
0    BIRK_01_500_000.jpeg
1    BIRK_01_500_001.jpeg
2    BIRK_01_500_002.jpeg
3    BIRK_01_500_003.jpeg
4    BIRK_01_500_004.jpeg
Name: image, dtype: object
Total images found in folder: 1819
All CSV images match with folder images!


In [8]:
# Sample images for train and val sets
sampled_images = random.sample(images, min(subset_size, len(images)))
num_train = len(sampled_images) // 2
train_images = sampled_images[:num_train]
val_images = sampled_images[num_train:]


In [9]:
# Helper function to convert XY coordinates to YOLO format
def convert_to_yolo(size, corners):
    img_w, img_h = size
    x_min = min([corners[i] for i in range(0, 8, 2)])
    x_max = max([corners[i] for i in range(0, 8, 2)])
    y_min = min([corners[i] for i in range(1, 8, 2)])
    y_max = max([corners[i] for i in range(1, 8, 2)])

    # Calculate YOLO bounding box (normalized)
    x_center = (x_min + x_max) / (2 * img_w)
    y_center = (y_min + y_max) / (2 * img_h)
    width = (x_max - x_min) / img_w
    height = (y_max - y_min) / img_h
    return 0, x_center, y_center, width, height  # class_id is 0 for "runway"

In [10]:
# Process images and generate labels
for subset, image_list in [('train', train_images), ('val', val_images)]:
    for img_file in image_list:
        img_path = os.path.join(image_folder, img_file)

        # Check if the image file exists
        if not os.path.exists(img_path):
            print(f"Image not found: {img_path}")
            continue  # Skip if the image doesn't exist

        # Load image and dimensions
        img = cv2.imread(img_path)
        if img is None:
            print(f"Failed to load image: {img_file}")
            continue  # Skip if image loading failed

        img_h, img_w = img.shape[:2]

        # Save processed image
        output_img_path = f'{output_folder}/images/{subset}/{img_file}'
        print(f"Saving {img_file} to {output_img_path}")
        cv2.imwrite(output_img_path, img)

        # Insert the updated label-saving block here
        # Correct label filename handling for both .jpg and .jpeg extensions
        base_filename, _ = os.path.splitext(img_file)  # Get the filename without the extension
        label_file = os.path.join(output_folder, 'labels', subset, base_filename + '.txt')

        print(f"Saving label for: {img_file}")
        print(f"Label will be saved as: {label_file}")

        try:
            # Ensure the metadata matches exactly
            csv_filenames = df['image'].str.strip().str.lower()
            img_file_normalized = img_file.strip().lower()

            # Find the corresponding row in the CSV
            row = df[csv_filenames == img_file_normalized].iloc[0]
            corners = row.iloc[-8:].values.astype(float)

            # Convert bounding box to YOLO format
            label_data = convert_to_yolo((img_w, img_h), corners)

            # Write the label file
            with open(label_file, 'w') as f:
                f.write(" ".join(map(str, label_data)) + '\n')

            print(f"Label saved successfully: {label_file}")

        except IndexError:
            print(f"No matching metadata found for image: {img_file}")


Saving LFST_23_500_444.jpeg to dataset_demo/images/train/LFST_23_500_444.jpeg
Saving label for: LFST_23_500_444.jpeg
Label will be saved as: dataset_demo\labels\train\LFST_23_500_444.txt
Label saved successfully: dataset_demo\labels\train\LFST_23_500_444.txt
Saving LFST_23_500_277.jpeg to dataset_demo/images/train/LFST_23_500_277.jpeg
Saving label for: LFST_23_500_277.jpeg
Label will be saved as: dataset_demo\labels\train\LFST_23_500_277.txt
Label saved successfully: dataset_demo\labels\train\LFST_23_500_277.txt
Saving BIRK_13_500_095.jpeg to dataset_demo/images/train/BIRK_13_500_095.jpeg
Saving label for: BIRK_13_500_095.jpeg
Label will be saved as: dataset_demo\labels\train\BIRK_13_500_095.txt
Label saved successfully: dataset_demo\labels\train\BIRK_13_500_095.txt
Saving BIRK_13_500_361.jpeg to dataset_demo/images/train/BIRK_13_500_361.jpeg
Saving label for: BIRK_13_500_361.jpeg
Label will be saved as: dataset_demo\labels\train\BIRK_13_500_361.txt
Label saved successfully: dataset_de

In [11]:
#SKIP FOR NOW WILL HaRDCOODE YAM FILE
# Create YAML configuration file
#yaml_content = f"""path: {output_folder}
#train: images/train
#val: images/val
#nc: 1
#names: ['runway']
#"""

In [12]:
#SKIP FOR NOW WILL HaRDCOODE YAM FILE
#with open(f'{output_folder}/runway.yaml', 'w') as f:
   # f.write(yaml_content)

#print("Data preparation complete!")

In [13]:
# Initialize YOLOv8 model and train
model = YOLO('yolov8n.pt')  # Use nano model for fast training
model.train(data=os.path.join(output_folder, 'runway.yaml'), epochs=10, imgsz=640)


Ultralytics 8.3.23  Python-3.11.9 torch-2.5.0+cpu CPU (AMD Ryzen 7 5800X 8-Core Processor)
engine\trainer: task=detect, mode=train, model=yolov8n.pt, data=dataset_demo\runway.yaml, epochs=10, time=None, patience=100, batch=16, imgsz=640, save=True, save_period=-1, cache=False, device=None, workers=8, project=None, name=train16, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=False, agnostic_nms=False, classes=None, retina_masks=False, embed=None, show=False, save_frames=False, save_txt=False, save_conf=False, save_crop=False, show_labels=True, show_conf=True, s

train: Scanning C:\Users\Annabelle\Desktop\runway-bounding-box-detection NEW\dataset_demo\labels\train... 10 images, 0 backgrounds, 0 corrupt: 100%|██████████| 10/10 [00:00<00:00, 1242.68it/s]

train: New cache created: C:\Users\Annabelle\Desktop\runway-bounding-box-detection NEW\dataset_demo\labels\train.cache



val: Scanning C:\Users\Annabelle\Desktop\runway-bounding-box-detection NEW\dataset_demo\labels\val... 10 images, 0 backgrounds, 0 corrupt: 100%|██████████| 10/10 [00:00<00:00, 4899.89it/s]

val: New cache created: C:\Users\Annabelle\Desktop\runway-bounding-box-detection NEW\dataset_demo\labels\val.cache


...

                   all         10         10      0.003        0.9    0.00354    0.00113



10 epochs completed in 0.010 hours.
Optimizer stripped from runs\detect\train16\weights\last.pt, 6.2MB
Optimizer stripped from runs\detect\train16\weights\best.pt, 6.2MB

Validating runs\detect\train16\weights\best.pt...
Ultralytics 8.3.23  Python-3.11.9 torch-2.5.0+cpu CPU (AMD Ryzen 7 5800X 8-Core Processor)
Model summary (fused): 168 layers, 3,005,843 parameters, 0 gradients, 8.1 GFLOPs


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 1/1 [00:00<00:00,  1.05it/s]


                   all         10         10      0.003        0.9    0.00355    0.00113
Speed: 1.2ms preprocess, 44.0ms inference, 0.0ms loss, 3.8ms postprocess per image
Results saved to runs\detect\train16


ultralytics.utils.metrics.DetMetrics object with attributes:

ap_class_index: array([0])
box: ultralytics.utils.metrics.Metric object
confusion_matrix: <ultralytics.utils.metrics.ConfusionMatrix object at 0x00000183341D9890>
curves: ['Precision-Recall(B)', 'F1-Confidence(B)', 'Precision-Confidence(B)', 'Recall-Confidence(B)']
curves_results: [[array([          0,    0.001001,    0.002002,    0.003003,    0.004004,    0.005005,    0.006006,    0.007007,    0.008008,    0.009009,     0.01001,    0.011011,    0.012012,    0.013013,    0.014014,    0.015015,    0.016016,    0.017017,    0.018018,    0.019019,     0.02002,    0.021021,    0.022022,    0.023023,
          0.024024,    0.025025,    0.026026,    0.027027,    0.028028,    0.029029,     0.03003,    0.031031,    0.032032,    0.033033,    0.034034,    0.035035,    0.036036,    0.037037,    0.038038,    0.039039,     0.04004,    0.041041,    0.042042,    0.043043,    0.044044,    0.045045,    0.046046,    0.047047,
          0.0480

In [14]:
# Evaluate the model
metrics = model.val()
print("Evaluation metrics:", metrics)

# Save the trained model
model.save('yolov8_runway_detector.pt')

# Test the model on a sample image (Optional)
test_image = r'C:\Users\Annabelle\Desktop\runway-bounding-box-detection NEW\test_image.jpeg' 
results = model.predict(source=test_image)
for r in results:
    r.show()
print("Training, evaluation, and testing complete!")

Ultralytics 8.3.23  Python-3.11.9 torch-2.5.0+cpu CPU (AMD Ryzen 7 5800X 8-Core Processor)
Model summary (fused): 168 layers, 3,005,843 parameters, 0 gradients, 8.1 GFLOPs


val: Scanning C:\Users\Annabelle\Desktop\runway-bounding-box-detection NEW\dataset_demo\labels\val.cache... 10 images, 0 backgrounds, 0 corrupt: 100%|██████████| 10/10 [00:00<?, ?it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 1/1 [00:00<00:00,  1.11it/s]


                   all         10         10      0.003        0.9    0.00355    0.00113
Speed: 1.4ms preprocess, 40.3ms inference, 0.0ms loss, 3.4ms postprocess per image
Results saved to runs\detect\train162
Evaluation metrics: ultralytics.utils.metrics.DetMetrics object with attributes:

ap_class_index: array([0])
box: ultralytics.utils.metrics.Metric object
confusion_matrix: <ultralytics.utils.metrics.ConfusionMatrix object at 0x0000018333FBB3D0>
curves: ['Precision-Recall(B)', 'F1-Confidence(B)', 'Precision-Confidence(B)', 'Recall-Confidence(B)']
curves_results: [[array([          0,    0.001001,    0.002002,    0.003003,    0.004004,    0.005005,    0.006006,    0.007007,    0.008008,    0.009009,     0.01001,    0.011011,    0.012012,    0.013013,    0.014014,    0.015015,    0.016016,    0.017017,    0.018018,    0.019019,     0.02002,    0.021021,    0.022022,    0.023023,
          0.024024,    0.025025,    0.026026,    0.027027,    0.028028,    0.029029,     0.03003,    0.03